In [ ]:
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import Laguerre_Gaussian as LG
import phaseScreenGeneration as PS
import CNN_Util as CU
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import h5py

In [ ]:
#Data metadata
modesUsed = 4
numTrainBatchPerMode = 4
batchSize = 50
numValBatchPerMode = 1
numBatchPerMode = 5

In [ ]:
#Data preprocessing
ML_BATCH_SIZE = 50
epochs = 10

train_data_gen = CU.batch_generator(idsT, ML_BATCH_SIZE, numTrainBatchPerMode,
                                 trainingDataList, trainingDataLabel, batchSize)

val_data_gen = CU.batch_generatorV(idsV, ML_BATCH_SIZE, numValBatchPerMode,
                                 validationDataList, validationDataLabel, batchSize)

In [ ]:
testModel = keras.Sequential([
    keras.layers.Conv2D(16 ,(5,5), activation = 'relu', padding='same', input_shape=(256,256,1)),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    #keras.layers.Conv2D(32, 3, activation = 'relu', padding='same'),
    #keras.layers.MaxPooling2D(),
    #keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    #keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(32, activation = 'softmax')
])

testModel.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

testModel.summary()

In [ ]:
history = testModel.fit_generator(
    train_data_gen,
    steps_per_epoch = np.ceil(len(idsT) / ML_BATCH_SIZE),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps= len(idsV) // ML_BATCH_SIZE
)

In [ ]:
CU.historyGraphs(history, range(epochs))

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'CNN_D&G/3x3medium4ModeCNN1History.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

testModel.save("CNN_D&G/3x3medium4ModeCNN1Model.h5")